# Midterm Activity 1

> An assumption for this activity is that your team was tasked to provide recommendations based on your descriptive and predictive analysis of the assigned dataset.

### Objectives

1. Provide a descriptive analysis of a given dataset
2. Interpret the results of association rule mining metrics.
3. Apply association rule mining as the basis for providing recommendations.

In [2]:
# Import the dataset
import pandas as pd

themepark = pd.read_csv('themepark.csv')

themepark

,Timestamp,Do you agree with the statements above?,What age bracket do you belong to?,What is your gender?,What is the most common reason for you to visit a theme park?,How often do you visit a theme park?,"Do you have a specific preference for visiting theme parks during certain seasons, or are you open to visiting throughout the year?",At what time of the week do you typically visit a theme park?,During which quarter of the year do you most prefer to visit theme parks?,Do you plan your theme park visits around school holidays and breaks?,...,How often do you visit each food choice? [Fast Food],How often do you visit each food choice? [Restaurants],How often do you visit each food choice? [Food Trucks or Snacks],"Aside from Food, what other attractions do you visit in theme parks?",What kinds of merchandise do you usually purchase when visiting a theme park?,"How often do you visit each shop? [Souvenirs (fridge magnets, keychains...)]","How often do you visit each shop? [Apparel (hats, shirts...)]","How often do you visit each shop? [Gift Shop (plushies, toys...)]","How often do you visit each shop? [Photo Services (photo booths, meet-and-greet photos...)]","Aside from Shopping, what other attractions do you visit in theme parks?"
0,4/3/2024 21:30:10,I agree,26-40,Male,"Vacation, Family Outing",Once a year,I am open to visiting throughout the year,Both,NaN,NaN,...,1.0,1.0,4.0,"Shopping and Merchandise (Gift Shops, Souvenir...","Souvenirs (fridge magnets, keychains...), Appa...",3.0,3.0,3.0,1.0,None anymore
1,4/3/2024 21:43:03,I agree,18-25,Male,Family Outing,Once a year,I prefer visiting during specific seasons,Both,4th Quarter (Oct - Dec),Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/3/2024 21:43:20,I agree,18-25,Female,"Vacation, Special Occasion, Date",2-3 times a year,I am open to visiting throughout the year,Weekdays,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/3/2024 21:43:43,I agree,18-25,Male,Family Outing,Once a year,I prefer visiting during specific seasons,Weekdays,2nd Quarter (Apr - Jun),No,...,3.0,5.0,5.0,None anymore,NaN,NaN,NaN,NaN,NaN,NaN
4,4/3/2024 21:50:47,I agree,18-25,Male,"Vacation, Family Outing",Once a year,I prefer visiting during specific seasons,Both,4th Quarter (Oct - Dec),Yes,...,4.0,5.0,4.0,"Entertainment and Shows (Parades, Fireworks, C...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,4/7/2024 20:29:52,I agree,Under 18,Female,Vacation,Once a year,I am open to visiting throughout the year,Weekends,NaN,NaN,...,5.0,4.0,5.0,"Shopping and Merchandise (Gift Shops, Souvenir...","Gift Shop (plushies, toys...)",3.0,3.0,5.0,3.0,"Entertainment and Shows (Parades, Fireworks, C..."
109,4/7/2024 23:15:33,I agree,56 and over,Female,"Vacation, Family Outing, Special Occasion,",Once a year,I prefer visiting during specific seasons,Weekends,4th Quarter (Oct - Dec),Yes,...,NaN,4.0,4.0,"Entertainment and Shows (Parades, Fireworks, C...","Souvenirs (fridge magnets, keychains...), Appa...",3.0,4.0,3.0,3.0,"Food and Dining (Fast Food, Restaurants, Snacks)"
110,4/8/2024 6:29:52,I agree,18-25,Female,"Vacation, Family Outing",2-3 times a year,I am open to visiting throughout the year,Weekends,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,4/8/2024 8:22:34,I agree,18-25,Female,"Vacation, Special Occasion",Once a year,I am open to visiting throughout the year,Weekends,NaN,NaN,...,NaN,NaN,NaN,NaN,"Souvenirs (fridge magnets, keychains...), Phot...",4.0,2.0,3.0,5.0,None anymore
